In [7]:
import os
import pandas as pd
import clickhouse_connect
from dotenv import load_dotenv

In [8]:
#truy nhập vào click house và lấy dữ liệu từ bảng tương ứng với điều kiện cụ thể
client = clickhouse_connect.get_client(
    host=os.getenv("CLICKHOUSE_HOST"),
    port=int(os.getenv("CLICKHOUSE_PORT")),
    username=os.getenv("CLICKHOUSE_USER"),
    password=os.getenv("CLICKHOUSE_PASSWORD")
)

query = """
SELECT *
FROM cem_network_test.result_full_flow_2025_31_12
WHERE delay_init_up > 0 and delay_init_up < 1500
"""

df = client.query_df(query)

FAIL_CODE = [1]
df['is_fail'] = df['cs_cause_code'].isin(FAIL_CODE)

TypeError: int() argument must be a string, a bytes-like object or a real number, not 'NoneType'

In [ ]:
# tính toán dữ liệu dựa trên các nhóm server_ip và time_window
agg = (
    df
    .groupby('server_ip', as_index=False)
    .agg(
        total_flow=('flow_idx', 'count'),
        total_failures=('is_fail', 'sum'),
        distinct_clients=('client_ip', 'nunique'),
        p_10_delayup=('delay_init_up', lambda x: x.quantile(0.10)),
        avg_delayup=('delay_init_up', 'mean')
    )
)

agg['failure_rate'] = agg['total_failures'] / agg['total_flow']
agg['stat_date'] = pd.to_datetime('2025-12-31')


In [33]:
agg.head()

,server_ip,total_flow,total_failures,distinct_clients,p_10_delayup,avg_delayup,failure_rate,stat_date
0,1.0.0.1,166,0,30,21.0,30.253012,0.0,2025-12-31
1,1.1.1.1,594,0,78,21.0,30.159933,0.0,2025-12-31
2,1.111.7.12,1,0,1,70.0,70.000000,0.0,2025-12-31
3,1.12.12.12,14,0,2,22.0,27.928571,0.0,2025-12-31
4,1.13.117.181,4,0,2,264.6,292.000000,0.0,2025-12-31


In [ ]:
#chuẩn hóa dữ liệu trước khi lưu vào clickhouse
agg['total_flow'] = agg['total_flow'].astype('uint32')
agg['total_failures'] = agg['total_failures'].astype('uint32')
agg['distinct_clients'] = agg['distinct_clients'].astype('uint32')

agg['p_10_delayup'] = agg['p_10_delayup'].astype('float32')
agg['avg_delayup'] = agg['avg_delayup'].astype('float32')
agg['failure_rate'] = agg['failure_rate'].astype('float32')


In [35]:
agg = agg[
    [
        'stat_date',
        'server_ip',
        'total_flow',
        'total_failures',
        'distinct_clients',
        'p_10_delayup',
        'avg_delayup',
        'failure_rate'
    ]
]


In [ ]:
#lưu dữ liệu vào database clickhouse
client.insert_df(
    table='cem_kpi.agg_server_daily',
    df=agg
)
